In [57]:
# 0.导包
# 0.1 pandas导入数据集
import pandas as pd
# 0.2 数据集分割
from sklearn.model_selection import train_test_split, GridSearchCV
# 0.3 数据标准化处理
from sklearn.preprocessing import StandardScaler
# 0.4 机器学习 KNN+CV
from sklearn.neighbors import KNeighborsClassifier

# 1.读取数据
original_train_data = pd.read_csv("../../Data/facebook-vpredicting-check-ins/train.csv")


In [58]:
# 2.基本数据处理
# 2.1 最小化数据集
train_data = original_train_data.query("x>2.0 & x < 2.5 & y>2.0 & y<2.5")
train_data

,row_id,x,y,accuracy,time,place_id
163,163,2.1663,2.3755,84,669737,3869813743
310,310,2.3695,2.2034,3,234719,2636621520
658,658,2.3236,2.1768,66,502343,7877745055
1368,1368,2.2613,2.3392,73,319822,9775192577
1627,1627,2.3331,2.0011,66,595084,6731326909
...,...,...,...,...,...,...
29116142,29116142,2.0804,2.0657,168,217886,1247398579
29116267,29116267,2.4309,2.4646,33,314360,1951613663
29116295,29116295,2.1797,2.1707,89,74954,4724115005
29116475,29116475,2.3924,2.2704,62,206257,2819110495


In [59]:
time

163        1970-01-08 18:02:17
310        1970-01-03 17:11:59
658        1970-01-06 19:32:23
1368       1970-01-04 16:50:22
1627       1970-01-07 21:18:04
                   ...        
29116142   1970-01-03 12:31:26
29116267   1970-01-04 15:19:20
29116295   1970-01-01 20:49:14
29116475   1970-01-03 09:17:37
29117203   1970-01-02 20:34:43
Name: time, Length: 70682, dtype: datetime64[ns]

In [ ]:
# 2.2 选择时间特征
time = pd.to_datetime(train_data["time"], unit='s')     # unit声明时间单位秒s
train_data["day"] = time.day
train_data["hour"] = time.hour
train_data["weekday"] = time.weekday

In [39]:
train_data.shape

(71664, 6)

In [40]:
time

163        1970-01-08 18:02:17
310        1970-01-03 17:11:59
658        1970-01-06 19:32:23
1368       1970-01-04 16:50:22
1627       1970-01-07 21:18:04
                   ...        
29116142   1970-01-03 12:31:26
29116267   1970-01-04 15:19:20
29116295   1970-01-01 20:49:14
29116475   1970-01-03 09:17:37
29117203   1970-01-02 20:34:43
Name: time, Length: 71664, dtype: datetime64[ns]

In [41]:
time = pd.DatetimeIndex(time)
time

DatetimeIndex(['1970-01-08 18:02:17', '1970-01-03 17:11:59',
               '1970-01-06 19:32:23', '1970-01-04 16:50:22',
               '1970-01-07 21:18:04', '1970-01-02 03:14:59',
               '1970-01-07 03:45:16', '1970-01-05 03:28:43',
               '1970-01-01 18:59:03', '1970-01-09 07:50:12',
               ...
               '1970-01-09 20:03:34', '1970-01-08 09:26:50',
               '1970-01-07 04:45:59', '1970-01-07 22:36:18',
               '1970-01-06 23:29:43', '1970-01-03 12:31:26',
               '1970-01-04 15:19:20', '1970-01-01 20:49:14',
               '1970-01-03 09:17:37', '1970-01-02 20:34:43'],
              dtype='datetime64[ns]', name='time', length=71664, freq=None)

In [42]:
time.hour

Index([18, 17, 19, 16, 21,  3,  3,  3, 18,  7,
       ...
       20,  9,  4, 22, 23, 12, 15, 20,  9, 20],
      dtype='int32', name='time', length=71664)

In [43]:
time.day

Index([8, 3, 6, 4, 7, 2, 7, 5, 1, 9,
       ...
       9, 8, 7, 7, 6, 3, 4, 1, 3, 2],
      dtype='int32', name='time', length=71664)

In [44]:
time.weekday

Index([3, 5, 1, 6, 2, 4, 2, 0, 3, 4,
       ...
       4, 3, 2, 2, 1, 5, 6, 3, 5, 4],
      dtype='int32', name='time', length=71664)

In [45]:
train_data

,row_id,x,y,accuracy,time,place_id
163,163,2.1663,2.3755,84,669737,3869813743
310,310,2.3695,2.2034,3,234719,2636621520
658,658,2.3236,2.1768,66,502343,7877745055
1368,1368,2.2613,2.3392,73,319822,9775192577
1627,1627,2.3331,2.0011,66,595084,6731326909
...,...,...,...,...,...,...
29116142,29116142,2.0804,2.0657,168,217886,1247398579
29116267,29116267,2.4309,2.4646,33,314360,1951613663
29116295,29116295,2.1797,2.1707,89,74954,4724115005
29116475,29116475,2.3924,2.2704,62,206257,2819110495


In [46]:
# 2.3 去除签到较少的地方
place_count = train_data.groupby("place_id").count()

In [47]:
place_count = place_count[place_count["row_id"] > 1]
place_count.head()

,row_id,x,y,accuracy,time
place_id,,,,,
1008823061,4,4,4,4,4
1012580558,3,3,3,3,3
1025585791,21,21,21,21,21
1026507711,220,220,220,220,220
1032417180,10,10,10,10,10


In [48]:
train_data = train_data[train_data["place_id"].isin(place_count.index)]
train_data.shape

(70682, 6)

In [49]:
# 2.4 选定特征值和目标值
x = train_data[["x", "y", "accuracy", "day", "hour", "weekday"]]
y = train_data["place_id"]
# 2.5 分割数据集
x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=20
)

# 3.特征工程——特征预处理（标准化）
# 3.1 实例化一个转换器
transfer = StandardScaler()
# 3.2 调用fit_transfer
x_train = transfer.fit_transform(x_train)
x_test = transfer.fit_transform(x_test)

# 4.机器学习 -- KNN+CV
# 4.1 实例化一个估计器
estimator = KNeighborsClassifier()
# 4.2 调用GridSearchCV
param_grid = {"n_neighbors": [1, 3, 5]}
estimator = GridSearchCV(estimator, param_grid, cv=0)
# 4.3 模型训练
estimator.fit(x_train, y_train)

KeyError: "['day', 'hour', 'weekday'] not in index"

In [ ]:
# 5.模型评估
# 5.1 基本评估方式
score = estimator.score(x_test, y_test)
print("最后预期的准确率为：", score)

y_predict = estimator.predict(x_test)
print("最后的预测值为：\n", y_predict)
print("预测值和真实值的对比情况：\n", y_predict == y_test)

In [ ]:
# 5.2 使用交叉验证后的评估方式
print("在交叉验证过程中最好的结果：\n", estimator.best_score_)
print("在交叉验证中最好的参数模型：\n", estimator.best_estimator_)
print("每次交叉验证后的验证集准确率结果和训练集准确率结果：\n", estimator.cv_results_)